# 100. 空間的CNNの実応用：ケーススタディ

## 学習目標

このノートブックでは、以下を学びます：

1. **医療画像解析**での空間的CNN
2. **自動運転**におけるセグメンテーション
3. **衛星画像解析**の応用
4. **3D再構成**とNeRF/3DGSへの接続

## 目次

1. [医療画像解析](#section1)
2. [自動運転](#section2)
3. [衛星画像解析](#section3)
4. [3D再構成への応用](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. 医療画像解析

### U-Netの誕生と医療分野への貢献

U-Netは2015年、医療画像セグメンテーションのために開発されました。

In [ ]:
def visualize_medical_applications():
    """医療画像解析の応用例を可視化"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    applications = [
        ('細胞セグメンテーション', 'U-Net（オリジナル）', '顕微鏡画像から細胞境界を検出'),
        ('腫瘍検出', 'CNN + セグメンテーション', 'CT/MRIから腫瘍領域を特定'),
        ('網膜血管検出', 'U-Net + Attention', '眼底画像から血管構造を抽出'),
        ('臓器セグメンテーション', '3D U-Net', 'CTスキャンから臓器を3D分割'),
        ('骨折検出', 'CNN + 分類', 'X線画像から骨折を検出'),
        ('皮膚病変分類', 'CNN + セグメンテーション', '皮膚画像から病変を分類'),
    ]
    
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral', 'lavender', 'lightpink']
    
    for ax, (title, method, desc), color in zip(axes.flat, applications, colors):
        ax.set_facecolor(color)
        ax.text(0.5, 0.7, title, ha='center', va='center', fontsize=14, fontweight='bold')
        ax.text(0.5, 0.45, f'手法: {method}', ha='center', va='center', fontsize=11)
        ax.text(0.5, 0.25, desc, ha='center', va='center', fontsize=10, wrap=True)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
    
    plt.suptitle('医療画像解析における空間的CNNの応用', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_medical_applications()

In [ ]:
def explain_medical_requirements():
    """医療画像解析の特有の要件"""
    print("="*60)
    print("医療画像解析の特有の要件")
    print("="*60)
    
    print("""
【データの特徴】
  ・データ量が少ない（アノテーションコストが高い）
  ・高解像度・高精度が必要
  ・3Dデータ（CT、MRI）が多い
  ・クラス不均衡（病変領域は小さい）

【なぜU-Netが適しているか】
  ・少ないデータでも学習可能
  ・スキップ接続で詳細な境界を保持
  ・データ拡張との相性が良い

【よく使われる損失関数】
  ・Dice Loss: 領域の重なりを最大化
  ・Focal Loss: 難しいサンプルに注目
  ・Combined Loss: 複数の損失を組み合わせ

【評価指標】
  ・Dice係数: 2|A∩B| / (|A| + |B|)
  ・IoU (Intersection over Union)
  ・Hausdorff距離（境界の精度）
    """)

explain_medical_requirements()

<a id="section2"></a>
## 2. 自動運転

In [ ]:
def visualize_autonomous_driving():
    """自動運転におけるセグメンテーションの可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 入力画像（模擬）
    ax = axes[0]
    img = np.ones((100, 150, 3)) * 0.7  # 空
    img[60:, :, :] = [0.3, 0.3, 0.3]  # 道路
    img[50:65, 60:90, :] = [0.8, 0.2, 0.2]  # 車
    img[55:70, 110:125, :] = [0.2, 0.6, 0.2]  # 人
    img[40:55, 10:25, :] = [0.4, 0.4, 0.2]  # 建物
    
    ax.imshow(img)
    ax.set_title('入力画像', fontsize=12)
    ax.axis('off')
    
    # セグメンテーション結果
    ax = axes[1]
    seg = np.zeros((100, 150, 3))
    seg[:60, :, :] = [0.5, 0.8, 1.0]  # 空
    seg[60:, :, :] = [0.3, 0.3, 0.5]  # 道路
    seg[50:65, 60:90, :] = [1.0, 0.2, 0.2]  # 車
    seg[55:70, 110:125, :] = [1.0, 1.0, 0.0]  # 人
    seg[40:55, 10:25, :] = [0.5, 0.5, 0.5]  # 建物
    
    ax.imshow(seg)
    ax.set_title('セマンティックセグメンテーション', fontsize=12)
    ax.axis('off')
    
    # クラス凡例
    ax = axes[2]
    classes = [
        ('空', [0.5, 0.8, 1.0]),
        ('道路', [0.3, 0.3, 0.5]),
        ('車両', [1.0, 0.2, 0.2]),
        ('歩行者', [1.0, 1.0, 0.0]),
        ('建物', [0.5, 0.5, 0.5]),
    ]
    
    for i, (name, color) in enumerate(classes):
        ax.add_patch(plt.Rectangle((0.1, 0.8 - i*0.15), 0.15, 0.1, facecolor=color))
        ax.text(0.35, 0.85 - i*0.15, name, fontsize=12, va='center')
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('クラス凡例', fontsize=12)
    
    plt.suptitle('自動運転におけるセマンティックセグメンテーション', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_autonomous_driving()

In [ ]:
def explain_autonomous_driving_pipeline():
    """自動運転のパイプライン"""
    print("="*60)
    print("自動運転における認識パイプライン")
    print("="*60)
    
    print("""
【主要タスク】
  1. セマンティックセグメンテーション
     → 道路、歩道、車線を識別
  
  2. インスタンスセグメンテーション
     → 個々の車両・歩行者を区別
  
  3. パノプティックセグメンテーション
     → セマンティック + インスタンスの統合
  
  4. 深度推定
     → 単眼カメラから距離を推定

【使用されるアーキテクチャ】
  ・DeepLab系: ASPP（Atrous Spatial Pyramid Pooling）
  ・PSPNet: Pyramid Pooling Module
  ・EfficientPS: パノプティック用
  ・SegFormer: Transformer + CNN

【リアルタイム性の要求】
  ・処理速度: 30fps以上
  ・レイテンシ: 100ms以下
  → 軽量モデル（MobileNet系バックボーン）が重要
    """)

explain_autonomous_driving_pipeline()

<a id="section3"></a>
## 3. 衛星画像解析

In [ ]:
def visualize_satellite_applications():
    """衛星画像解析の応用"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    applications = [
        ('土地利用分類', '森林、農地、都市、水域などを分類\n\n使用手法:\n- U-Net + ResNetバックボーン\n- マルチスペクトル画像対応'),
        ('建物検出', '衛星画像から建物のフットプリントを抽出\n\n使用手法:\n- Mask R-CNN\n- インスタンスセグメンテーション'),
        ('変化検出', '2時点間の変化（森林伐採、都市化）を検出\n\n使用手法:\n- Siamese Network\n- 差分画像解析'),
        ('災害被害評価', '洪水、火災、地震の被害範囲を特定\n\n使用手法:\n- セマンティックセグメンテーション\n- 時系列解析'),
    ]
    
    colors = ['lightgreen', 'lightblue', 'lightyellow', 'lightcoral']
    
    for ax, (title, desc), color in zip(axes.flat, applications, colors):
        ax.set_facecolor(color)
        ax.text(0.5, 0.85, title, ha='center', va='center', fontsize=14, fontweight='bold')
        ax.text(0.5, 0.45, desc, ha='center', va='center', fontsize=10)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
    
    plt.suptitle('衛星画像解析における空間的CNNの応用', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_satellite_applications()

In [ ]:
def explain_satellite_challenges():
    """衛星画像解析の課題"""
    print("="*60)
    print("衛星画像解析の特有の課題")
    print("="*60)
    
    print("""
【データの特徴】
  ・超高解像度（数万×数万ピクセル）
  ・マルチスペクトル（RGB以外のバンドも利用）
  ・時系列データ（季節変化、経年変化）
  ・雲、影、大気の影響

【技術的課題と解決策】

  1. 巨大画像の処理
     → タイル分割 + オーバーラップ
     → 境界のスムージング
  
  2. スケール変化
     → FPN（Feature Pyramid Network）
     → マルチスケール推論
  
  3. クラス不均衡
     → Focal Loss
     → サンプリング戦略
  
  4. ドメインシフト
     → 地域ごとのファインチューニング
     → ドメイン適応
    """)

explain_satellite_challenges()

<a id="section4"></a>
## 4. 3D再構成への応用

### CNNから3DGSへの架け橋

In [ ]:
def visualize_3d_reconstruction_pipeline():
    """3D再構成パイプラインの可視化"""
    fig, ax = plt.subplots(figsize=(18, 8))
    
    # パイプラインのステップ
    steps = [
        (0.05, 0.5, '入力画像群', 'lightblue'),
        (0.22, 0.5, '特徴抽出\n(CNN)', 'lightgreen'),
        (0.39, 0.5, '対応点検出\n(マッチング)', 'lightyellow'),
        (0.56, 0.5, 'カメラ推定\n(SfM)', 'lightcoral'),
        (0.73, 0.5, '3D表現\n(NeRF/3DGS)', 'lavender'),
        (0.90, 0.5, 'レンダリング', 'lightpink'),
    ]
    
    for x, y, label, color in steps:
        rect = plt.Rectangle((x, y - 0.15), 0.12, 0.3, 
                            facecolor=color, edgecolor='gray', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + 0.06, y, label, ha='center', va='center', fontsize=10)
    
    # 矢印
    for i in range(len(steps) - 1):
        ax.annotate('', xy=(steps[i+1][0], steps[i+1][1]),
                   xytext=(steps[i][0] + 0.12, steps[i][1]),
                   arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    # CNNの役割を強調
    ax.annotate('', xy=(0.28, 0.25), xytext=(0.28, 0.35),
               arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.text(0.28, 0.15, 'CNNの帰納バイアス\n（局所性・重み共有）\nが特徴抽出を効率化', 
           ha='center', fontsize=10, color='red')
    
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('3D再構成パイプラインとCNNの役割', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_3d_reconstruction_pipeline()

In [ ]:
def explain_cnn_in_3d_reconstruction():
    """3D再構成におけるCNNの役割"""
    print("="*60)
    print("3D再構成におけるCNNの役割")
    print("="*60)
    
    print("""
【特徴抽出での活用】
  ・SuperPoint: キーポイント検出（CNN）
  ・SuperGlue: 特徴マッチング（GNN + Attention）
  ・LoFTR: Detector-freeマッチング（Transformer + CNN）

【深度推定での活用】
  ・MiDaS: 単眼深度推定
  ・RAFT-Stereo: ステレオマッチング
  → CNN + コスト集約で高精度な深度マップ

【NeRF/3DGSでの活用】
  ・CNN特徴を3D表現の初期化に利用
  ・画像エンコーダとしてのCNN（PixelNeRF等）
  ・3DGSの初期点群生成にSfM（CNN特徴ベース）

【NeoVerseプロジェクトへの接続】
  ・このノートブックシリーズで学んだCNNの知識が
    3DGS/NeRFの理解の基盤となる
  ・受容野の概念 → 3DGSのスプラットサイズの理解
  ・帰納バイアス → なぜ3D表現が効率的か
    """)

explain_cnn_in_3d_reconstruction()

In [ ]:
def compare_2d_to_3d():
    """2D CNNと3D表現の比較"""
    print("="*70)
    print("2D CNN vs 3D表現（NeRF/3DGS）")
    print("="*70)
    
    comparison = [
        ('', '2D CNN', '3DGS/NeRF'),
        ('-'*15, '-'*25, '-'*25),
        ('入力', '画像（2D）', '画像群（複数視点）'),
        ('出力', '特徴マップ/セグメンテーション', '3Dシーン表現'),
        ('表現', '畳み込みカーネル', 'ガウシアン/ニューラル場'),
        ('帰納バイアス', '局所性・重み共有', 'マルチビュー一貫性'),
        ('受容野', '固定（設計時に決定）', '適応的（学習で最適化）'),
        ('計算コスト', '比較的低い', '高い（レンダリング必要）'),
    ]
    
    for row in comparison:
        print(f"{row[0]:<15} {row[1]:<25} {row[2]:<25}")

compare_2d_to_3d()

<a id="summary"></a>
## 5. まとめ

### 空間的CNNの応用分野

| 分野 | 主要タスク | キーアーキテクチャ |
|------|----------|------------------|
| 医療画像 | セグメンテーション | U-Net, 3D U-Net |
| 自動運転 | 認識・セグメンテーション | DeepLab, PSPNet |
| 衛星画像 | 土地利用分類 | U-Net + FPN |
| 3D再構成 | 特徴抽出・深度推定 | SuperPoint, MiDaS |

### 次のステップ

次のノートブックでは、これまで学んだ内容を総合的に振り返り、空間的帰納バイアスの本質を深く理解します。

In [ ]:
def final_summary():
    """最終まとめ"""
    print("="*70)
    print("空間的CNNの応用 - まとめ")
    print("="*70)
    
    print("""
【学んだこと】

1. 医療画像解析
   ・U-Netが医療分野で革命を起こした
   ・少ないデータでも高性能を達成
   ・Dice Loss等の専用損失関数が重要

2. 自動運転
   ・セマンティック/インスタンス/パノプティックセグメンテーション
   ・リアルタイム性が重要な制約
   ・軽量モデルの必要性

3. 衛星画像解析
   ・超高解像度画像の処理
   ・マルチスペクトル・時系列データ
   ・タイル分割とFPNの活用

4. 3D再構成
   ・CNNは特徴抽出・深度推定の基盤
   ・NeRF/3DGSへの橋渡し
   ・NeoVerseプロジェクトとの接続

【重要な洞察】
  CNNの空間的帰納バイアスは、
  画像処理から3D理解まで幅広く応用される
  普遍的な設計原理である。
    """)

final_summary()